<font color="Green"><h1> **Week 4 Mini-Project Part 2**</h1></font>

In [1]:
#import neccesary libraries and functions

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error
from scipy.spatial.distance import cdist
from scipy.sparse import coo_matrix
from sklearn.decomposition import NMF

In [2]:
#Load the data

MV_users = pd.read_csv('data/users.csv')
MV_movies = pd.read_csv('data/movies.csv')
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

from collections import namedtuple
Data = namedtuple('Data', ['users','movies','train','test'])
data = Data(MV_users, MV_movies, train, test)

In [3]:
#Borrowed code from the week 3 assignment

class rating_matrix_generator():
    def __init__(self,data):
        self.data=data
        self.allusers = list(self.data.users['uID'])
        self.allmovies = list(self.data.movies['mID'])
        self.genres = list(self.data.movies.columns.drop(['mID', 'title', 'year']))
        self.mid2idx = dict(zip(self.data.movies.mID,list(range(len(self.data.movies)))))
        self.uid2idx = dict(zip(self.data.users.uID,list(range(len(self.data.users)))))
        self.Mr=self.rating_matrix()
        
    def rating_matrix(self):
        """
        Convert the rating matrix to numpy array of shape (#allusers,#allmovies)
        """
        ind_movie = [self.mid2idx[x] for x in self.data.train.mID] 
        ind_user = [self.uid2idx[x] for x in self.data.train.uID]
        rating_train = list(self.data.train.rating)
        
        return np.array(coo_matrix((rating_train, (ind_user, ind_movie)), shape=(len(self.allusers), len(self.allmovies))).toarray())


In [4]:
#generate a rating_matrix_obj object to get the rating matrix

rating_matrix_obj = rating_matrix_generator(data)

#run our NMF model

NMF_model = NMF(n_components = 5, init = 'nndsvd')
W = NMF_model.fit_transform(rating_matrix_obj.Mr)
H = NMF_model.components_

In [5]:
#double check and make sure our matrix dimensions are as we expect them to be

print(W)
print(W.shape)

print(H)
print(H.shape)

[[0.         0.03225928 0.11321427 0.23750959 0.13005165]
 [0.37912879 0.1596561  0.19872182 0.         0.14686068]
 [0.0530025  0.01571611 0.07719231 0.08189181 0.21062493]
 ...
 [0.         0.04498333 0.0087366  0.03835132 0.06075382]
 [0.         0.47691515 0.         0.16367524 0.14318159]
 [0.         1.00416528 0.40369362 0.01390889 0.13276606]]
(6040, 5)
[[0.         0.81931961 0.38412165 ... 0.04772011 0.         0.11776665]
 [0.16913315 0.         0.         ... 0.08647346 0.01744    0.11006545]
 [1.41964834 0.         0.         ... 0.04744123 0.06635167 0.74422576]
 [2.11188132 0.78037394 0.75868578 ... 0.         0.         0.        ]
 [1.24972038 0.25353795 0.         ... 0.         0.         0.        ]]
(5, 3883)


In [6]:
#define a function to predict a given user-movie combination based on our decomposed matrices

def predict(uID, mID, W, H):
    user_index = rating_matrix_obj.uid2idx[uID]
    movie_index = rating_matrix_obj.mid2idx[mID]
    u = W[user_index]
    m = H[:,movie_index] 
    pred_rating = u.dot(m)
    
    return pred_rating

In [7]:
#run our predictions on the test data

y_pred = []
for rating in range(len(test)):
    y_pred.append(predict(test['uID'][rating], test['mID'][rating], W, H))
    

In [8]:
#use sklearn's RMSE funciton
print(mean_squared_error(test['rating'], y_pred, squared = True))

8.948641757546483


<font color="Green"><h1> **Supervised vs. Unsupervised discussion**</h1></font>

As evidenced by our RMSE result, the NMF model for predicting user rating on a user-rating matrix does not work out so well. On epossible way to fix the issue would be to use categorical embeddings for our data so that when we run the matrix, similar to how we did in the BBC news article competition.